In [ ]:
""""
    def solve_ode_Leapfrog(self, t_pts, r_0, r_dot_0, phi_0):

        #Solve the ODE given initial conditions with the Leapfrog method.

        delta_t = t_pts[1] - t_pts[0]
        
        # initialize the arrays for r, rdot, r_dot_half, phi with zeros
        num_t_pts = len(t_pts)
        r = np.zeros(num_t_pts)
        r_dot = np.zeros(num_t_pts)
        r_dot_half = np.zeros(num_t_pts)
        phi = np.zeros(num_t_pts)
        
        # initial conditions
        r[0] = r_0
        r_dot[0] = r_dot_0
        phi[0] = phi_0
        
        # step through the differential equation
        for i in np.arange(num_t_pts - 1):
            t = t_pts[i]
            y = [r[i], r_dot[i], phi[i]]
            r_dot_half[i] = r_dot[i] + self.dy_dt(t, y)[1] * delta_t/2.
            r[i+1] = r[i] + r_dot_half[i] * delta_t
            
            y = [r[i+1], r_dot[i], phi[i]]
            r_dot[i+1] = r_dot_half[i] + self.dy_dt(t, y)[1] * delta_t/2.
            
            phi[i+1] = phi[i] + self.dy_dt(t,y)[2] * delta_t
        return r, r_dot, phi   
        
    
    def energy(self, t_pts, r, r_dot):
        #Evaluate the energy as a function of time

        return (self.mu/2.) * r_dot**2 + self.Ueff(r)
""""

#x1_pts_LF, x2_pts_LF, y1_pts_LF, y2_pts_LF \
#                           = o1.solve_ode_Leapfrog(t_pts, x1, x2, y1, y2)

c = o1.ang_mom**2 / (np.abs(o1.k) * o1.mu)
epsilon = c / r_0 - 1.
energy_0 = o1.mu/2. * r_dot_0**2 + o1.Ueff(r_0)
print(f'energy = {energy_0:.2f}')
print(f'eccentricity = {epsilon:.2f}')


# Gravitational orbits in Cartesian coordinates

Make a notebook that solves the two-body problem for gravitational
attraction between two bodies in Cartesian coordinates. This will 
require x and y coordinates for both bodies and the use of the 
Lagrangian method. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [6]:
class Orbit:
    
    def __init__(self, G=1, m1=1, m2=2):
        # Defining subjective conditions for m1 and m2. 
        self.G = G
        self.m1 = m1
        self.m2 = m2
        
    def dz_dt(self, t, z):
        """
        Parameters
        ----------
        t : float
            time 
        z : float
            There are eight equations in total as seen below.
            z[0] = x1(t) and z[1] = x_dot_1(t),
            z[2] = y1(t) and z[3] = y_dot_1(t),
            z[4] = x2(t) and z[5] = x_dot_2(t),
            z[6] = y2(t) and z[7] = y_dot_2(t),
        r_12 : Pontential energy for gravitional force
        """
        r_12 = np.sqrt((z[0]-z[4])**2 + (z[2]-z[6])**2)
        return [z[1], self.G * self.m1 * (z[4] - z[0]) / r_12**3,\
                z[3], (self.G * self.m2 * (-z[2] + z[6]) / r_12**3),\
                z[5], self.G * self.m1 * (-z[4] + z[0]) / r_12**3,\
                z[7], (self.G * self.m1 * (-z[6] + z[2])) / r_12**3,]
    
    
    def solve_ode(self, t_pts, x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot,
                  method='RK23',
                  abserr=1.0e-8, relerr=1.0e-8):
        """
        Solve the ODE given initial conditions.
        Use solve_ivp with the option of specifying the method.
        Specify smaller abserr and relerr to get more precision.
        """
        z = [x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot]  
        solution = solve_ivp(self.dz_dt, (t_pts[0], t_pts[-1]), 
                             z, t_eval=t_pts, method=method, 
                             atol=abserr, rtol=relerr)
        x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot = solution.z
        return x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot

In [7]:
def start_stop_indices(t_pts, plot_start, plot_stop):
    start_index = (np.fabs(t_pts-plot_start)).argmin()  # index in t_pts array 
    stop_index = (np.fabs(t_pts-plot_stop)).argmin()  # index in t_pts array 
    return start_index, stop_index

In [ ]:
# Initial Conditions for G, mass 1, and mass 2
G=1.
m1=1.
m2=2.
o1 = Orbit(G=G, m1=m1, m2=m2)

# Plotting time 
t_start = 0.
t_end = 10.
delta_t = 0.001/10

t_pts = np.arange(t_start, t_end+delta_t, delta_t)  

# Initial conditions
x1 = 1.
x2 = 1.
y1 = 1. 
y2 = 1.
x1_dot = 1. 
x2_dot = 2.
y1_dot = 2.
y2_dot = 1.
x1_pts, x1_dot_pts, y1_pts, y1_dot_pts, x2_pts, x2_dot_pts, y2_pts, y2_dot_pts = o1.solve_ode(t_pts, x1, x1_dot, y1, y1_dot, x2, x2_dot, y2, y2_dot)
"""
# Graphing pts for both x and y
ax_4a = fig_4.add_subplot(2,1,1)
ax_4a.plot(x1_pts, y1_pts, color='black', label='RK23')
#ax_4a.plot(t_pts, r_pts_Euler, color='blue', label='Euler')
#ax_4a.plot(t_pts, r_pts_LF, color='red', label='Leapfrog')
ax_4a.set_xlabel(r'$y$')
ax_4a.set_ylabel(r'$x$')
ax_4a.set_title(r'$x1_pts versus y1_pts$')
ax_4a.legend()

ax_4b = fig_4.add_subplot(2,1,2)
ax_4b.plot(x2_pts, y2_pts/(2.*np.pi), color='black', label='RK23')
ax_4b.set_xlabel(r'$x$')
ax_4b.set_ylabel(r'$y$')
ax_4b.set_title(r'$x2_pts versus y2_pts$')
ax_4b.legend()
"""

In [ ]:
E_tot_pts = o1.energy(t_pts, r_pts, r_dot_pts)
E_tot_0 = E_tot_pts[0]
E_tot_rel_pts = np.abs((E_tot_pts - E_tot_0)/E_tot_0)

""""
E_tot_pts_LF = o1.energy(t_pts, r_pts_LF, r_dot_pts_LF)
E_tot_0_LF = E_tot_pts_LF[0]
E_tot_rel_pts_LF = np.abs((E_tot_pts_LF - E_tot_0_LF)/E_tot_0_LF)
""""

In [ ]:
fig_5 = plt.figure(figsize=(6,6))

overall_title = 'Orbit:  ' + \
                rf' $n = {o1.n},$' + \
                rf' $k = {o1.k:.1f},$' + \
                rf' $l = {o1.ang_mom:.1f},$' + \
                rf' $r_0 = {r_0:.1f},$' + \
                rf' $\dot r_0 = {r_dot_0:.1f},$' + \
                rf' $\phi_0 = {phi_0:.1f}$' + \
                rf' $delta:t = 0.001/10$' + \
                '\n'     # \n means a new line (adds some space here)
fig_5.suptitle(overall_title, va='baseline')

ax_5a = fig_5.add_subplot(1,1,1)
#ax_5a.semilogy(t_pts, np.abs(E_tot_pts), color='black', label=r'$E(t)$')
ax_5a.semilogy(t_pts, E_tot_rel_pts, 
               color='green', label=r'$\Delta E(t)$ RK23')
ax_5a.semilogy(t_pts, E_tot_rel_pts_Euler, 
               color='blue', label=r'$\Delta E(t)$ Euler')
ax_5a.semilogy(t_pts, E_tot_rel_pts_LF, 
               color='red', label=r'$\Delta E(t)$ Leapfrog')
ax_5a.set_ylim(1.e-10, 1.e-2)    # (1.e-12, 5)
ax_5a.set_xlabel(r'$t$')
ax_5a.set_ylabel(r'Energy')
ax_5a.set_title('Change in energy with time')
ax_5a.legend()

fig_5.tight_layout()
fig_5.savefig('Leapfrog_energy_test_Final_project.png', dpi=200, bbox_inches='tight')